# Baseline

Reference Link: https://www.analyticsvidhya.com/blog/2021/10/human-pose-estimation-using-machine-learning-in-python/

In [92]:
# Preparation
import mediapipe as mp
import cv2
import time
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # For drawing keypoints
points = mpPose.PoseLandmark # Landmarks
path = "dataset/train/"
data = []
for p in points:
        x = str(p)[13:]
        data.append(x + "_x")
        data.append(x + "_y")
        data.append(x + "_z")
        data.append(x + "_vis")
data = pd.DataFrame(columns = data) # Empty dataset

In [93]:
# Creating Dataset
target = []
count = 0

for subdir, dirs, files in os.walk(path):
    for img in files:
        temp = []
        img = os.path.join(subdir, img)
        img = cv2.imread(img)

        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        blackie = np.zeros(img.shape) # Blank image
        results = pose.process(imgRGB)

        if results.pose_landmarks:
                mpDraw.draw_landmarks(blackie, results.pose_landmarks, mpPose.POSE_CONNECTIONS) # draw landmarks on blackie
                landmarks = results.pose_landmarks.landmark

                for i,j in zip(points,landmarks):
                        temp = temp + [j.x, j.y, j.z, j.visibility]
                data.loc[count] = temp
                target.append(subdir.replace(path, ''))
                count +=1

data['target'] = target

Premature end of JPEG file
Corrupt JPEG data: premature end of data segment


In [94]:
# Label Encoding for target
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
data['target'] = labelencoder.fit_transform(data['target'])

In [95]:
# Displaying Dataset
data.head(5)

,NOSE_x,NOSE_y,NOSE_z,NOSE_vis,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_z,LEFT_EYE_INNER_vis,LEFT_EYE_x,LEFT_EYE_y,...,RIGHT_HEEL_vis,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,LEFT_FOOT_INDEX_z,LEFT_FOOT_INDEX_vis,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_z,RIGHT_FOOT_INDEX_vis,target
0,0.385088,0.702528,-0.004816,0.999651,0.364045,0.705285,-0.031445,0.999706,0.361666,0.700772,...,0.525770,0.781881,0.930616,-0.215838,0.980343,0.763475,0.904605,0.165073,0.610637,0
1,0.470336,0.691998,-0.604218,0.982091,0.445842,0.705398,-0.624718,0.987435,0.437711,0.699783,...,0.526152,0.778034,0.569747,0.541724,0.935673,0.764064,0.639336,0.467184,0.604327,0
2,0.453251,0.615995,-0.057232,0.983838,0.440873,0.630020,-0.067001,0.988656,0.440118,0.630212,...,0.552748,0.769751,0.797690,0.004431,0.937697,0.743715,0.761387,0.285678,0.625030,0
3,0.401504,0.383240,-0.309374,0.984927,0.436412,0.378602,-0.318324,0.989338,0.404689,0.379723,...,0.591222,0.622063,0.713210,-0.035523,0.916048,0.569005,0.763766,-0.045453,0.652138,0
4,0.450490,0.683425,-0.067524,0.986200,0.446467,0.690085,-0.090036,0.990196,0.426712,0.690337,...,0.598638,0.656271,0.870245,0.077734,0.911403,0.668724,0.867453,0.307724,0.658130,0


## Exploratory Data Analysis

In [96]:
data.describe(include = 'all')

,NOSE_x,NOSE_y,NOSE_z,NOSE_vis,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_z,LEFT_EYE_INNER_vis,LEFT_EYE_x,LEFT_EYE_y,...,RIGHT_HEEL_vis,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,LEFT_FOOT_INDEX_z,LEFT_FOOT_INDEX_vis,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_z,RIGHT_FOOT_INDEX_vis,target
count,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,...,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000
mean,0.505858,0.415518,-0.246159,0.943101,0.509002,0.405122,-0.245095,0.946450,0.511065,0.404905,...,0.733155,0.540911,0.875051,0.117718,0.757308,0.438541,0.860734,0.123024,0.729785,2.058394
std,0.169495,0.206963,0.314906,0.048036,0.175483,0.211302,0.313211,0.045594,0.176174,0.210777,...,0.114746,0.254270,0.421723,0.458191,0.109323,0.252702,0.413459,0.467372,0.117507,1.456656
min,-0.515029,-0.864020,-2.438290,0.698547,-0.572157,-0.926884,-2.470850,0.713646,-0.586003,-0.926810,...,0.300710,-0.409310,-0.324660,-1.693826,0.305906,-0.555812,-0.288427,-1.124857,0.287977,0.000000
25%,0.430587,0.251077,-0.403654,0.921806,0.434221,0.235587,-0.384058,0.923609,0.433971,0.235401,...,0.664399,0.342956,0.715532,-0.141796,0.706594,0.252431,0.702412,-0.131425,0.656494,1.000000
50%,0.497861,0.397042,-0.207230,0.960206,0.503867,0.384749,-0.199818,0.962283,0.506907,0.381391,...,0.743797,0.547789,0.849946,0.050389,0.778800,0.421433,0.840961,0.052675,0.742752,2.000000
75%,0.565261,0.563995,-0.048967,0.976734,0.568594,0.563723,-0.059908,0.978745,0.572143,0.563686,...,0.819670,0.747333,0.937381,0.307297,0.832774,0.599478,0.931384,0.297200,0.820857,3.000000
max,1.226446,1.296077,1.270609,0.999651,1.227893,1.272271,1.262069,0.999706,1.236210,1.268909,...,0.934545,1.339510,5.438205,3.869035,0.980343,1.537790,5.224499,3.975790,0.939650,4.000000


In [97]:
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt


label_df = pd.DataFrame()
label_df['label'] = list(map(lambda x: labelencoder.inverse_transform([x])[0], data['target']))

bars = alt.Chart(label_df).mark_bar(size=50).encode(
    x=alt.X('label', axis=alt.Axis(title='Pose')),
    y=alt.Y("count()", axis=alt.Axis(title='Count')),
    tooltip=[alt.Tooltip('count()', title='Count'), 'label'],
    color='label'
)

(bars).interactive().properties(
    height=300, 
    width=700,
    title = "Number of data in each pose",
)


alt.Chart(...)

In [98]:
# Traning the baseline model with SVM
from sklearn.svm import SVC
X,Y = data.iloc[:,:132],data['target']
model = SVC(kernel = 'poly')
model.fit(X,Y)

SVC(kernel='poly')

In [99]:
# Predicting test 
test_path = "dataset/test/"
y_pred = []
y_test = []
for subdir, dirs, files in os.walk(path):
    for img in files:
        temp = []
        img = os.path.join(subdir, img)
        img = cv2.imread(img)

        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        blackie = np.zeros(img.shape) # Blank image
        results = pose.process(imgRGB)

        if results.pose_landmarks:
                mpDraw.draw_landmarks(blackie, results.pose_landmarks, mpPose.POSE_CONNECTIONS) # draw landmarks on blackie
                landmarks = results.pose_landmarks.landmark

                for i,j in zip(points,landmarks):
                        temp = temp + [j.x, j.y, j.z, j.visibility]
                y_pred.append(model.predict([temp]))
                y_test.append(labelencoder.transform([subdir.replace(path, '')])[0])

Premature end of JPEG file
Corrupt JPEG data: premature end of data segment


In [100]:
# Evaluating the baseline model (SVM)
from sklearn.metrics import classification_report

target_names = labelencoder.classes_
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     downdog       0.99      0.97      0.98       196
     goddess       0.93      0.84      0.88       164
       plank       0.94      0.98      0.96       225
        tree       0.97      0.85      0.91       136
    warrior2       0.86      0.96      0.91       238

    accuracy                           0.93       959
   macro avg       0.94      0.92      0.93       959
weighted avg       0.93      0.93      0.93       959



# Implementing YOLOX

In [21]:
import cv2
from pathlib import Path

import numpy as np
import torch
from torch import nn
from torch.nn import functional as F

from yolox.data_augment import preproc
from yolox.yolox import YOLOX, get_model, IdentityModule

In [2]:
# YOLOX Configuration
class dotdict(dict):
    """
    Dotdict is just a dictionary whose elements can be referenced with a dot operation.
    I.e. dotdict['x'] == dotdict.x

    This is useful because the original YOLOX used a custom class to hold a lot of extra configuration that
    we do not need.
    """
    def __getattr__(self, x):
        return self['x']


opt = dotdict()
# All images should be scaled to this input size before passing through YOLOX.
# Any image (of any size) can be scaled using the function `yolox.data_augment.preproc`
# I don't recommend changing this. This is just fine and loads pretty quickly, even on CPU.
opt.input_size = (640, 640)
opt.random_size = (10, 20)  # None; multi-size train: from 448(14*32) to 832(26*32), set None to disable it
opt.test_size = (640, 640)
opt.rgb_means = [0.485, 0.456, 0.406]
opt.std = [0.229, 0.224, 0.225]
opt.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
opt.backbone = "CSPDarknet-nano"
opt.depth_wise = True
opt.use_amp = False  # True, Automatic mixed precision

In [25]:
from typing import List


# Load YOLOX (Including weights pretrained on COCO)

# The head (i.e. the connection between the YOLOX backbone and neck to the rest of the model) is by default just an IdentityModule.
# This head should be exchanged with some torch module that performs the rest of the function (in this case classification)
# The head module should be a torch module expecting an input that is a list of 3 tensors of sizes:
#        [torch.Size([BATCH_SIZE, 64, 80, 80]), torch.Size([BATCH_SIZE, 128, 40, 40]), torch.Size([BATCH_SIZE, 256, 20, 20])]
# Note: These sizes may change if the `opt.input_size` or `opt.test_size` are changed.
# Each of these inputs is a different output of the YOLOX neck and represents the features learned at various scales.

# The YOLOX model expects a single tensor input of size: [BATCH_SIZE, 3, opt.test_size[0], opt.test_size[1]]
# BATCHSIZE is the Batch size
# 3 is the number of color channels (the YOLOX is pretrained on 3 channels. Even if the image is grayscale, convert it to RGB
# opt.test_size[0] is the number of horizontal pixels in the input
# opt.test_size[1] is the number of vertical pixels in the input

class ClassificationHead(nn.Module):
    def __init__(self, input_sizes:List[int], input_channels:List[int], num_classes:int, hidden_features:int = 128):
        super(ClassificationHead, self).__init__()
        self.fc0a = nn.Linear(input_channels[0]*input_sizes[0]**2,hidden_features)
        self.fc0b = nn.Linear(input_channels[1]*input_sizes[1]**2,hidden_features)
        self.fc0c = nn.Linear(input_channels[2]*input_sizes[2]**2,hidden_features)
        # Concatenate the three outputs into one linear layer
        self.fc1 = nn.Linear(len(input_sizes) * hidden_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, hidden_features)
        self.fc3 = nn.Linear(hidden_features, num_classes)

    def forward(self,x):
        a = F.relu(self.fc0a(torch.flatten(x[0],1)))
        b = F.relu(self.fc0b(torch.flatten(x[1],1)))
        c = F.relu(self.fc0c(torch.flatten(x[2],1)))
        x = torch.cat([a,b,c], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.softmax(self.fc3(x), dim=1)
        x = self.fc3(x)
        return x

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eps = 1e-3
                m.momentum = 0.03


model = get_model(opt,
                  # head=IdentityModule(),
                  head=ClassificationHead([80,40,20], [64,128,256], 5),
                  freeze_layers=True)

# Check if frozen
assert not any(p.requires_grad for p in model.backbone.parameters())

==>> loaded pretrained_models/yolox-nano.pth, epoch 294
--> Drop parameter head.stems.0.conv.weight.
--> Drop parameter head.stems.0.bn.weight.
--> Drop parameter head.stems.0.bn.bias.
--> Drop parameter head.stems.0.bn.running_mean.
--> Drop parameter head.stems.0.bn.running_var.
--> Drop parameter head.stems.0.bn.num_batches_tracked.
--> Drop parameter head.stems.1.conv.weight.
--> Drop parameter head.stems.1.bn.weight.
--> Drop parameter head.stems.1.bn.bias.
--> Drop parameter head.stems.1.bn.running_mean.
--> Drop parameter head.stems.1.bn.running_var.
--> Drop parameter head.stems.1.bn.num_batches_tracked.
--> Drop parameter head.stems.2.conv.weight.
--> Drop parameter head.stems.2.bn.weight.
--> Drop parameter head.stems.2.bn.bias.
--> Drop parameter head.stems.2.bn.running_mean.
--> Drop parameter head.stems.2.bn.running_var.
--> Drop parameter head.stems.2.bn.num_batches_tracked.
--> Drop parameter head.cls_convs.0.0.dconv.conv.weight.
--> Drop parameter head.cls_convs.0.0.dco

In [5]:
# Load Images
img_dir = 'imgs/'
images = [cv2.imread(str(im)) for im in Path(img_dir).glob('*.jpg')]
print(f'There are {len(images)} images')
inp_imgs = np.zeros([len(images), 3, opt.test_size[0], opt.test_size[1]], dtype=np.float32)
for b_i, image in enumerate(images):
    img, r = preproc(image, opt.test_size, opt.rgb_means, opt.std)
    inp_imgs[b_i] = img

inp_imgs = torch.from_numpy(inp_imgs).to(opt.device)
print(f'Input image batch of shape: {inp_imgs.shape}')

There are 2 images
Input image batch of shape: torch.Size([2, 3, 640, 640])


In [24]:
# Run inference as a test to make sure network runs.
with torch.no_grad():
    yolo_outputs = model(inp_imgs)
    # print(yolo_outputs)
    print(yolo_outputs.shape)

torch.Size([2, 10])


## TODO: Implement a custom training loop
As long as `get_model` is called with `freeze_layers=True`, the early layers (the YOLOX pretrained ones) will be frozen, so training should be fast--only the head needs to be trained.

In [ ]:
# Load Images
while not stopping_condition:
    # Take input
    # Send it through the model
    # Calculate the loss
    # Run back propagation
    pass


